In [1]:
import networkx as nx
import warnings
import numpy as np
import os
from pandas import DataFrame
import csv
import os.path
from causalnex.structure import StructureModel
from causalnex.plots import plot_structure
from numpy import array, save, load
from networkx import to_numpy_matrix
from cdt.causality.graph import CAM
import cdt
from pandas import DataFrame
from numpy import float32
from os import path
warnings.filterwarnings("ignore")  # silence warnings

# cdt.SETTINGS.rpath = os.getenv("RSCRIPT_PATH")  # path to your r executable
cdt.SETTINGS.rpath = 'C:\Program Files\R\R-4.2.1\\bin\Rscript' # path to your r executable



Detecting 1 CUDA device(s).


In [2]:
DATA_DIR = "CLeaR_2023_Dataset"
DATA_SPARSE_DIR = os.path.join(DATA_DIR, "sparse")
DATA_DENSE_DIR = os.path.join(DATA_DIR, "dense")
data_dirs = [DATA_SPARSE_DIR, DATA_DENSE_DIR]

In [3]:
method_dirs = ['linear_mechanism', 'mix_mechanism']

# Plot graphs

In [4]:
def generate_plot(dag_path, plot_path, csv_path):
    dag = np.load(dag_path)

    graph = nx.from_numpy_array(dag, create_using=nx.DiGraph)

    e = list(graph.edges())
    causal_nex_graph = StructureModel(e)
    viz = plot_structure(causal_nex_graph)  # Default CausalNex visualisation
    viz.draw(plot_path, format="jpg")

    # check if the file exists
    file_exists = os.path.isfile(csv_path)

    # open the file in append mode
    with open(csv_path, "a", newline="") as csvfile:
        # create a CSV writer object
        writer = csv.writer(csvfile)

        # if the file doesn't exist, write the header row
        if not file_exists:
            writer.writerow(["nodes", "edges"])

        # write the value to the CSV file
        writer.writerow([graph.number_of_nodes(), graph.number_of_edges()])

In [5]:
def plot_data_dir(data_dir):
    for meth_dir in method_dirs:
        method_path = os.path.join(data_dir, meth_dir)
        print(method_path)
        mechanism_dirs = os.listdir(path=method_path)

        for dataset_dir in mechanism_dirs:
            dataset_path = os.path.join(method_path, dataset_dir)
            dag_plot_path = os.path.join(dataset_path, "plot")
            if not os.path.isdir(dag_plot_path):
                os.mkdir(dag_plot_path)

            dag_confounder_path = os.path.join(dataset_path, "confounder_DAG1.npy")
            dag_path = os.path.join(dataset_path, "DAG1.npy")

            # Plot Paths
            dag_confounder_plot_path = os.path.join(dag_plot_path, "confounder_plot.jpg")
            dag_plot_path = os.path.join(dag_plot_path, "plot.jpg")
            csv_path = os.path.join(dataset_path, "details.csv")
            generate_plot(dag_path, dag_plot_path, csv_path)
            if path.exists(dag_confounder_path):
                generate_plot(dag_confounder_path, dag_confounder_plot_path, csv_path)


In [6]:
plot_data_dir(DATA_SPARSE_DIR)

CLeaR_2023_Dataset\sparse\linear_mechanism
CLeaR_2023_Dataset\sparse\mix_mechanism


In [6]:
plot_data_dir(DATA_DENSE_DIR)

CLeaR_2023_Dataset\dense\linear_mechanism
CLeaR_2023_Dataset\dense\mix_mechanism


# Run CAM on data

In [7]:
def run_cam(data: array, output_path: str):
    print("=================")
    print("Running CAM: ", output_path)
    print("=================")
    cam_result_dir = os.path.join(output_path, "cam")
    cam_result_path = os.path.join(cam_result_dir, "result.npy")
    if not os.path.isdir(cam_result_dir):
        os.mkdir(cam_result_dir)
    obj = CAM()
    df = DataFrame(data).astype(float32)
    output = obj.predict(df)
    pred = to_numpy_matrix(output)
    save(cam_result_path, pred)

In [8]:
data_mix_mechanism_path = os.path.join(DATA_SPARSE_DIR, "mix_mechanism")
dag_path = os.path.join(data_mix_mechanism_path, "small_mixed_all_issues_3")
dag_data_path = os.path.join(dag_path, "data1.npy")
data = load(dag_data_path)
data

array([[-0.79925409, -0.71223992, -1.42566298, ..., -1.54151049,
         0.85435523,  2.68873163],
       [-0.73175804, -1.18817698, -1.06359189, ..., -0.49245175,
         0.64567766,  1.39755125],
       [-1.35430492, -1.38586543, -1.11022287, ..., -1.74689566,
         0.53072395,  4.35447961],
       ...,
       [ 0.6343002 ,  1.467998  ,  2.52825811, ...,  0.37776338,
         0.4535749 ,  0.33155417],
       [-0.04587649, -1.25319933,  2.5070255 , ..., -2.24164148,
         0.34519461, -0.41713075],
       [-1.03954153,  0.14134816,  2.71774188, ..., -0.68268577,
        -0.16745526, -1.00698656]])

In [9]:
run_cam(data, "")

Running CAM:  
